<a href="https://colab.research.google.com/github/day-ch/Appteste/blob/main/O_que_se_tem_estudado_sobre_videogames_e_o_espectro_autista%3F_Programa%C3%A7%C3%A3o_em_Phython_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## O que se tem estudado sobre videogames e o espectro autista? Uma revisão sistemática de dados da base pubmed com auxílio de processamento de linguagem natural e deep learning.




Este notebook traz o código em Phython utilizado para a elaboração da revisão sistemática de dados do pubmed "O que se tem estudado sobre videogames e o espectro autista: uma revisão sistemática de dados da base pubmed com auxílio de processamento de linguagem natural e deep learning", apresentado no formato de poster no VII Congresso Internacional e XXVII Congresso Nacional da ABENEPI (Associação Brasileira de Neurologia e Psiquiatria Infantil e Profissões Afins).

Os códigos podem ser utilizados em auxílio a pesquisas e revisões de outros temas na base pubmed.


1) DOWNLOAD DOS ABSTRACTS


> Inicialmente o tema de interesse é buscado na base pubmed e os dados de cada artigo encontrado são baixados e salvos em um tabela no formato '.csv'. É gerado um arquivo com os artigos e seus 'abstracts'.

> Busca-se pelos termos pretendidos no pubmed, após a busca copia-se o que aparece no campo de endereço eletrônico após o "term=" e cola esse código no paramêtro "query" do código abaixo.

> O código abaixo copia todas as informações da busca para uma tabela .csv que fica salva na aba 'arquivos' do colab.

> Cada linha da tabela corresponde a um artigo, com colunas para título do artigo, autores, informações de autores, revista, 'abstract', DOI e miscelânea.

> O algoritmo abaixo utiliza o pacote 'eutils 0.6.0' para esses procedimentos.



Para mais informações sobre os parâmetros e o uso do pacote eutils para baixar informações do pubmed favor consultar:

 Sayers E. The E-utilities In-Depth: Parameters, Syntax and More. 2009 May 29 [Updated 2022 Nov 30]. In: Entrez Programming Utilities Help [Internet]. Bethesda (MD): National Center for Biotechnology Information (US); 2010-. Available from: https://www.ncbi.nlm.nih.gov/books/NBK25499/https://www.ncbi.nlm.nih.gov/books/NBK25499/

 Sayers E. E-utilities Quick Start. 2008 Dec 12 [Updated 2018 Oct 24]. In: Entrez Programming Utilities Help [Internet]. Bethesda (MD): National Center for Biotechnology Information (US); 2010-. Available from: https://www.ncbi.nlm.nih.gov/books/NBK25500/









In [ ]:
import csv
import re
import urllib
from time import sleep

# inicialmente busca-se as palavras chaves e/ou termos no campo de busca do pubmed
#(no caso da nossa revisão: ("Autism Spectrum Disorder"[Mesh]) AND "Video Games"[Mesh])), clica em 'Search'
# vai aparecer no campo de endereço eletrônico :
# https://pubmed.ncbi.nlm.nih.gov/?sort=date&term=(%22Autism+Spectrum+Disorder%22%5BMesh%5D)+AND+%22Video+Games%22%5BMesh%5D
# copia tudo que aparece após 'term=', (%22Autism+Spectrum+Disorder%22%5BMesh%5D)+AND+%22Video+Games%22%5BMesh%5D
# coloca como o parâmetro query abaixo

query = '(("Autism+Spectrum+Disorder"%5BMesh%5D)++AND+"Video+Games"%5BMesh%5D'
# common settings between esearch and efetch
base_url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
db = 'db=pubmed'

# esearch settings
search_eutil = 'esearch.fcgi?'
search_term = '&term=' + query
search_usehistory = '&usehistory=y'
search_rettype = '&rettype=json'

# call the esearch command for the query and read the web result
search_url = base_url+search_eutil+db+search_term+search_usehistory+search_rettype
print("this is the esearch command:\n" + search_url + "\n")
f = urllib.request.urlopen (search_url)
search_data = f.read().decode('utf-8')

# extract the total abstract count
total_abstract_count = int(re.findall("<Count>(\d+?)</Count>",search_data)[0])

# efetch settings
fetch_eutil = 'efetch.fcgi?'
retmax = 20
retstart = 0
fetch_retmode = "&retmode=text"
fetch_rettype = "&rettype=abstract"

# obtain webenv and querykey settings from the esearch results
fetch_webenv = "&WebEnv=" + re.findall ("<WebEnv>(\S+)<\/WebEnv>", search_data)[0]
fetch_querykey = "&query_key=" + re.findall("<QueryKey>(\d+?)</QueryKey>",search_data)[0]

# call efetch commands using a loop until all abstracts are obtained
run = True
all_abstracts = list()
loop_counter = 1

while run:
    print("this is efetch run number " + str(loop_counter))
    loop_counter += 1
    fetch_retstart = "&retstart=" + str(retstart)
    fetch_retmax = "&retmax=" + str(retmax)
    # create the efetch url
    fetch_url = base_url+fetch_eutil+db+fetch_querykey+fetch_webenv+fetch_retstart+fetch_retmax+fetch_retmode+fetch_rettype
    print(fetch_url)
    # open the efetch url
    f = urllib.request.urlopen (fetch_url)
    fetch_data = f.read().decode('utf-8')
    # split the data into individual abstracts
    abstracts = fetch_data.split("\n\n\n")
    # append to the list all_abstracts
    all_abstracts = all_abstracts+abstracts
    print("a total of " + str(len(all_abstracts)) + " abstracts have been downloaded.\n")
    # wait 2 seconds so we don't get blocked
    sleep(2)
    # update retstart to download the next chunk of abstracts
    retstart = retstart + retmax
    if retstart > total_abstract_count:
        run = False

In [ ]:
with open("Abstracts_total1.csv", "wt",encoding="utf-8") as abstracts_file:
    abstract_writer = csv.writer(abstracts_file)
    abstract_writer.writerow(['Journal', 'Title', 'Authors', 'Author_Information', 'Abstract', 'DOI', 'Misc'])
    #For each abstract, split into categories and write it to the csv file
    for abstract in all_abstracts:
        #To obtain categories, split every double newline.
        split_abstract = abstract.split("\n\n")
        abstract_writer.writerow(split_abstract)

2) APLICAÇÃO DE CRITÉRIOS DE INCLUSÃO E EXCLUSÃO

A tabela gerada no passo anterior é manualmente checada. Os artigos que não se encaixam nos critérios de inclusão ou aqueles que apresentam critérios para exclusão são excluídos da tabela.


> Critérios de inclusão: População que engloba crianças, adolescentes ou adultos com transtorno do espectro autista (TEA). Artigos sobre uso de video games em português, inglês ou espanhol. Artigos dos últimos 20 anos.


> Critérios de exclusão: População que não TEA. Outras línguas que não português, inglês ou espanhol. Artigos que não tem como assunto video games. Artigos sem abstracts disponíveis.

A tabela gerada após aplicação dos critérios de elegibilidade se encontra abaixo:

https://drive.google.com/file/d/1ZQ_Gi8gS768-Zvj7t_fx2h-bepTvpG1a/view?usp=drive_link



3) SUMARIZAÇÃO DOS ABSTRACTS UTILIZANDO DEEP LEARNING

Os abstracts são sumarizados de forma automatizada utilizando um modelo de deep learning de arquitetura 'transformers' da google o 'Pegasus x-sum'. Os abstracts sumarizados em uma frase são salvos em um arquivo/tabela .csv.



In [ ]:
import pandas as pd

In [ ]:
#leitura de csv com titulo e abstract
df=pd.read_csv('', usecols= ['Title','Abstract'])
print(df)

In [ ]:
df['Text']=df["Title"].map(str) + " " + df["Abstract"]
text=df['Text'].tolist()
text2=[]
for item in text:
    item2=str(item)
    text2.append(item2)

In [ ]:
!pip install transformers
!pip install transformers[sentencepiece]
!pip install torch

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

src_text = text2

model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

summary=[]
for text in src_text:
  batch = tokenizer(text, truncation=True, padding="longest", return_tensors="pt").to(device)
  translated = model.generate(**batch)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  summary.append(tgt_text)

In [ ]:
myFile = open('Sumario.csv', 'w')
writer = csv.writer(myFile)
writer.writerow(['Sumario'])
for data_list in summary:
    writer.writerow(data_list)
myFile.close()
myFile = open('Sumario.csv', 'r')
print("The content of the csv file is:")
print(myFile.read())
myFile.close()

4) PROCESSAMENTO DE LINGUAGEM NATURAL E FREQUÊNCIAS

A tabela gerada no passo 2 é pré-processada, sendo acrescentadas as colunas "Ano" e "Journals", conforme link abaixo:

https://drive.google.com/file/d/1qrShMjjZhyfqnvGwGX0A4KrTnW5Z8eDa/view?usp=drive_link

São utilizadas técnicas de processamento de linguagem natural aplicadas as colunas da tabela dos artigos. São calculados as frequências das palavras na coluna Autores para auxiliar na identificação dos autores com o maior número de publicações sobre o tema. O mesmo é feito para a coluna 'Ano','Journal', 'Informação de autores' e 'abstracts'.

A partir da frequência de palavras de todo o texto contido em todas as linhas da coluna 'abstract' (com a exclusão de stopwords definidas) é montado uma nuvem de palavras que pode apontar para temas mais investigados pelos artigos pubmed sobre video games e o espectro autista.



In [1]:
import pandas as pd
dr = pd.read_csv('/content/drive/MyDrive/COMPARTILHADA Poster Abenepi/Artigos_Revisão3.csv')

FREQUÊNCIA AUTORES

In [ ]:
dr["mytext_new"] = dr['Authors'].str.lower().str.replace('[^\w\s]','').str.replace('\n', ' ').str.replace('[^A-Za-z0-9]+', ' ')
dr['Authors'] = dr['mytext_new'].str.replace('\d+', '')

new_df = dr.mytext_new.str.split(expand=True).stack().value_counts().reset_index()
new_df.columns = ['Word', 'Frequency']

mask = (new_df['Word'].str.len() > 3)
dy = new_df.loc[mask]
dy

FREQUÊNCIA ANO

In [ ]:
ano= dr['Ano'].value_counts()
print(ano)

FREQUÊNCIA REVISTA

In [ ]:
dfj= dr['Journals'].value_counts()
print(dfj)

FREQUÊNCIA DE PALAVRAS EM ABSTRACTS

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Baixar o corpus de Stopwords
nltk.download('stopwords')

# Baixar o tokenizador
nltk.download('punkt')

# Função para pré-processar o texto
def preprocess_text(text):
    # Converter para minúscula
    text = text.lower()

    # Remover pontuações
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenizar o texto
    tokens = word_tokenize(text)

    # Remover as stopwords
    stop_words = stopwords.words('english')
    new_stopwords = ["game","games","video","years","asd", "autism", "spectrum","research","disorders","analysis","people","time","studies","findings","control","significant","disorder","analisys","methods","also","associated","disorder", "study","results", "use","individuals","used","data","information","autistic","may","group", "participants", "using"]
    stop_words.extend(new_stopwords)
    tokens = [token for token in tokens if token not in stop_words]

    # Juntar os tokens em um 'string'
    text = ' '.join(tokens)

    return text

# Aplicar a função de pré-processamento de texto e criar uma coluna com os abstracts processados
dr['AbstractPre'] = dr['Abstract'].apply(preprocess_text)

# Visualização das cinco primeiras linhas do dataframe pós-processamento
print(dr.head())

In [ ]:
from collections import Counter

# Criando frequência de palavras
word_freq = Counter()

for text in dr['AbstractPre']:
    word_freq.update(text.split())

# Visualizar palavras mais frequentes
print(word_freq.most_common(30))

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Gerar uma nuvem de palavras
wordcloud = WordCloud(width=800, height=800, background_color='black').generate_from_frequencies(word_freq)

# Plotar uma nuvem de palavras
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

In [ ]:
# create image as cloud
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
# store to file
plt.savefig("cloud.png", format="png")
plt.show()

In [ ]:
from os import path
from PIL import Image
import numpy as np
import os

mask = np.array(Image.open("figura"))
mask=np.where(mask > 3, 255, mask)
wordcloud = WordCloud(background_color='white',mask=mask,contour_color='#023075',contour_width=3,colormap='rainbow',min_font_size=10).generate_from_frequencies(word_freq)


# create image as cloud
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
# store to file
plt.savefig("cloud.png", format="png")
plt.show()